In [14]:
import pandas as pd
import numpy
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_pickle('bikeshare.pkl')

In [3]:
df.columns

Index(['Date', 'Rented Bike Count', 'Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day'],
      dtype='object')

In [7]:
X,y = df[['Hour','Temperature(°C)']], df['Rented Bike Count']

In [10]:
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10)

In [12]:
lr = LinearRegression()

In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cross_val_score(lm, X, y, cv=kf, scoring='r2')

array([0.41899504, 0.38464178, 0.41885967, 0.40057989, 0.4187674 ])

In [16]:
np.std(cross_val_score(lm, X, y, cv=kf, scoring='r2'))

array([0.41899504, 0.38464178, 0.41885967, 0.40057989, 0.4187674 ])